<a href="https://colab.research.google.com/github/preetsanjam/Crypto_Price_Analysis/blob/main/Sentiment_Ethereum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import mwclient
import time
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings('ignore')

site = mwclient.Site('en.wikipedia.org')
page = site.pages['Ethereum']

In [ ]:
revs = list(page.revisions())

In [ ]:
revs[0]

OrderedDict([('revid', 1257117784),
             ('parentid', 1252608296),
             ('user', 'HmmInterdasting'),
             ('timestamp',
              time.struct_time(tm_year=2024, tm_mon=11, tm_mday=13, tm_hour=9, tm_min=36, tm_sec=24, tm_wday=2, tm_yday=318, tm_isdst=-1)),
             ('comment', '/* Accounts */ Message calls are not transactions')])

In [ ]:
revs = sorted(revs, key=lambda rev:rev['timestamp'])

In [ ]:
revs[0]

OrderedDict([('revid', 592567939),
             ('parentid', 0),
             ('user', 'Sanpitch'),
             ('timestamp',
              time.struct_time(tm_year=2014, tm_mon=1, tm_mday=27, tm_hour=1, tm_min=53, tm_sec=45, tm_wday=0, tm_yday=27, tm_isdst=-1)),
             ('comment',
              "[[WP:AES|←]]Created page with '{{Infobox currency | image_1 =  | image_title_1 =  | image_width_1 =  | image_2 =  | image_title_2 =  | image_width_2 =  |issuing_authority = None. The Ethereum...'")])

In [ ]:
from transformers import pipeline
sentiment_pipleine = pipeline('sentiment-analysis', model='distilbert/distilbert-base-uncased-finetuned-sst-2-english', revision='714eb0f')

def find_sentiment(text):
  sent = sentiment_pipleine(text[:250])[0]
  score = sent['score']
  if sent['label'] == 'NEGATIVE':
    score *= -1
  return score

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
edits = {}

for rev in revs:
    date = time.strftime("%Y-%m-%d", rev["timestamp"])
    if date not in edits:
        edits[date] = dict(sentiments=list(), edit_count=0)

    edits[date]["edit_count"] += 1

    comment = rev.get("comment", "")
    edits[date]["sentiments"].append(find_sentiment(comment))

In [ ]:
from statistics import mean

for key in edits:
    if len(edits[key]["sentiments"]) > 0:
        edits[key]["sentiment"] = mean(edits[key]["sentiments"])
        edits[key]["neg_sentiment"] = len([s for s in edits[key]["sentiments"] if s < 0]) / len(edits[key]["sentiments"])
    else:
        edits[key]["sentiment"] = 0
        edits[key]["neg_sentiment"] = 0

    del edits[key]["sentiments"]

In [ ]:
import pandas as pd

edits_df = pd.DataFrame.from_dict(edits, orient="index")

In [ ]:
edits_df

,edit_count,sentiment,neg_sentiment
2014-01-27,1,-0.998511,1.000000
2014-02-01,1,-0.997276,1.000000
2014-04-06,5,0.790979,0.000000
2014-04-09,24,0.646407,0.083333
2014-04-10,9,-0.361518,0.666667
...,...,...,...
2024-09-09,1,0.930021,0.000000
2024-09-29,1,0.994775,0.000000
2024-10-09,1,-0.999581,1.000000
2024-10-22,1,-0.999592,1.000000


In [ ]:
edits_df.index = pd.to_datetime(edits_df.index)

In [ ]:
from datetime import datetime

dates = pd.date_range(start="2014-01-27",end=datetime.today())

In [ ]:
edits_df = edits_df.reindex(dates, fill_value=0)

In [ ]:
edits_df

,edit_count,sentiment,neg_sentiment
2014-01-27,1,-0.998511,1.0
2014-01-28,0,0.000000,0.0
2014-01-29,0,0.000000,0.0
2014-01-30,0,0.000000,0.0
2014-01-31,0,0.000000,0.0
...,...,...,...
2024-11-26,0,0.000000,0.0
2024-11-27,0,0.000000,0.0
2024-11-28,0,0.000000,0.0
2024-11-29,0,0.000000,0.0


In [ ]:
rolling_edits = edits_df.rolling(30, min_periods=30).mean()

In [ ]:
rolling_edits = rolling_edits.dropna()

In [ ]:
rolling_edits

,edit_count,sentiment,neg_sentiment
2014-02-25,0.066667,-0.066526,0.066667
2014-02-26,0.033333,-0.033243,0.033333
2014-02-27,0.033333,-0.033243,0.033333
2014-02-28,0.033333,-0.033243,0.033333
2014-03-01,0.033333,-0.033243,0.033333
...,...,...,...
2024-11-26,0.033333,-0.033160,0.033333
2024-11-27,0.033333,-0.033160,0.033333
2024-11-28,0.033333,-0.033160,0.033333
2024-11-29,0.033333,-0.033160,0.033333


In [ ]:
rolling_edits.to_csv("ethereum_wikipedia_edits.csv")